# Calibration of Corona model
This workbook shows how a parameter in the corona model specified [here](Corona%20specify%20model%20and%20make%20eksperiments.ipynb) can be calibrated to match one of the output variables.

The notebook will run when loaded. You can change the cells and run again by press run.  

In [333]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd

from modelclass import model
from modelinvert import targets_instruments
model.modelflow_auto()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and run
The model and baseline run has been saved in [here](Corona%20specify%20model%20and%20make%20eksperiments.ipynb)

In [334]:
mcorona,df  = model.modelload('coronatest.json')  # Get the model and data
df.index = df.index-100                           # rebase the days, so we calibrate at time 0
df.index.name = 'Day'
res = mcorona(df,keep='Basis',silent=1)           # run the baseline 

In [335]:
def ti_calibrate(mmodel,instrument,target,value,time,delay=0,silent=1,show=1,keep_restore= False):
    mmodel.keep_solutions,old_keep_solutions= {},mmodel.keep_solutions
    mmodel.keep_solutions['Before calibration'] = mmodel.lastdf.copy(deep=True)
    
    targetdf=pd.DataFrame(value,index=[time],columns=[target])
    ti = targets_instruments(databank=mmodel.lastdf,model=mmodel,targets=targetdf,instruments= [instrument])
    
    res = ti(mmodel.lastdf,delay=delay,silent=1,debug=1)
    res2 = mmodel(res,silent=silent,keep='After calibration')
    if show:
        figs = mcorona.keep_plot(f'{instrument[0]} {target}',diff=0,legend=0,dec='2',showtype='level',scale='linear');
        mmodel.keep_add_vline(figs,time=time) 
    if keep_restore:
        mmodel.keep_solutions,model_calibrate_keep_solutions =  old_keep_solutions,mmodel.keep_solutions
    return res2

# Now calibrate the parameter to match an infectious growth rate of 3 percent 

You can try with different values or variables and the pres the run button. 

In [336]:
target_value = 3   # the value we want to hit 
time         = 0   # the day we want to calibrate
delay        = 5   # the delay in effect

res = mcorona(df,keep='Basis',silent=1)
res2 = ti_calibrate(mcorona,instrument = ('PROBABILITY_TRANSMISION',0.01),
                    target = 'INFECTIOUS_GROWTH',value=target_value,
                    time=time,delay=delay,show=False)

# And plot the results 

In [337]:
figs = mcorona.keep_viznew('dead hospital_icu infectious')

Output()

In [338]:
list(mcorona.keep_solutions.keys())[0]


'Before calibration'

In [339]:
from ipywidgets import widgets
ii= widgets.SelectMultiple(value=['2'],options=[r'\(\color{red} {highlighted}dd\)','2','3'],rows=2)
display(ii)

SelectMultiple(index=(1,), options=('\\(\\color{red} {highlighted}dd\\)', '2', '3'), rows=2, value=('2',))

In [340]:
ii.keys

['_dom_classes',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_options_labels',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'description',
 'description_tooltip',
 'disabled',
 'index',
 'layout',
 'rows',
 'style']

In [341]:
ii.rows

2